# CASO DE NEGOCIO: Modelamiento de Retención de Clientes en Scholastic Travel Company

## Sinopsis

Scholastic Travel Company (STC) quiere usar los datos de sus clientes para predecir quién comprará un paquete de viaje el próximo año. Los casos presentan datos de 2.389 clientes, donde el conjunto A contiene varios campos de perfil y el conjunto B contiene información adicional del Net Promoted Score (NPS)

Este notebook ha sido preparado con información provista en el Caso de Negocios de Harvard UV7579 (Agosto 23, 2018), Retention Modeling at Scholastic Travel Company, elaborado por Anton Ovchinnikov, profesor de Management Science, Operations Management y Customer Analytics, en la escuela de negocios Smith, de Queen’s University en Canadá.

## Objetivos

Los objetivos del ejercicio son:

1.   Entender el problema de negocio presentado y cómo este problema se convierte en un problema de analítica de datos
2.   Aplicar técnicas adecuadas en preparación de datos para clasificación
3.   Aplicar técnicas adecuadas en limpieza de datos: lidiar con valores faltantes, categorías extrañas
4.   Practicar con herramientas avanzadas de analítica de datos como regresión logística (incluyendo la selección de variables con StepAIC or Recusrsive Feature Elimination para Python) y máquinas de vectores de soporte (SVM)
5.   Entender las métricas de casificación más comunes: matriz de confusión, curva ROC, Área bajo la curva AUC, para poder comparar modelos analíticos

Los principales pasos a seguir son:

1.   Asegurar que los paquetes y librerías necesarios están instalados (e instalarlos si no lo están)
2.   Cargar los paquetes y librerías necesarios
3.   Cargar los datos
4.   "Limpiar" los datos: necesitaremos (4.1) convertir algunas características en tipos correctos, (4.2) combinar categorías de tasas, (4.3) arreglar los valores que faltan, y (4.4) crear dummies (one hot encoding) para las características no numéricas
5.   Definir el objetivo (la variable que intentamos predecir) y la matriz de características (todas las demás, excepto el ID)
6.   Dividir los datos en entrenamiento y prueba
7.   Entrenar (ajustar) el modelo con los datos de entrenamiento 
8.   Aplicarlo a los datos de prueba 
9.   Calcular las métricas del modelo y ajustar los hiperparámetros para mejorar esas métricas
10.  Exportar las predicciones para la toma de decisiones


Consideraremos varios modelos de analítica de datos:

1. REGRESIÓN LOGÍSTICA: una gneeralización de la regresión -- un modelo que, como la regresión lineal, sigue teniendo coeficientes, valores p y similares, pero aplica transformaciones especiales para tener en cuenta el hecho de que predice categorias en vez de cantidades continuas

2. ÁRBOLES DE DECISIÓN - CART: un modelo que es conceptualmente diferente de la regresión, ya que no tiene coeficientes. Veremos que este método es útil para la visualización y la comprensión, pero no es muy preciso para las predicciones

3. MÁQUINAS DE VECTORES DE SOPORTE: un método intermedio entre las regresiones y los árboles que se ajusta a planos de menor dimensión para separar los datos en clases con el máximo margen entre ellas

4. 

5.  

### Paso 0: Para iniciar ... 

In [1]:
# Activar múltiples hilos en su computador, para un cálculo más rápido 
%env OMP_NUM_THREADS = 4

env: OMP_NUM_THREADS=4


# Pasos 1 y 2: Instalar y cargar los paquetes y librerías necesarias

In [2]:
# Paso 1: Chequear el ambiente Conda y los paquetes/librerías instalados
# import sys
# !conda env list
# !conda list
# !conda update --all

# Descargar e instalar pandas, numpy, scikit-learn. Podría ser necesario hacerlos desde el prompt de Anaconda
# !conda install pandas # pandas includes numpy 
# !conda install scikit-learn

# Paso 2: Cargar los paquetes y librerías necesarios 

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")

# Paso 3: Cargar los datos

In [33]:
# Paso 3: Cargar los datos desde el archivo CSV en el dataframe llamado df.

df = pd.read_csv(r'/Users/aramirez/Documents/Analitica 2/Caso de Negocio/04 CSV data -- STC(A)_numerical dates.csv', header = 0, delimiter=';', decimal='.', na_values='NaN', keep_default_na=True)  
df.head() # show the "head" -- first 5 rows of the data; note, these are rows 0...4

,ID,Program.Code,From.Grade,To.Grade,Group.State,Is.Non.Annual.,Days,Travel.Type,Departure.Date,Return.Date,...,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,SingleGradeTripFlag,FPP.to.School.enrollment,FPP.to.PAX,Num.of.Non_FPP.PAX,SchoolSizeIndicator,Retained.in.2012.
0,1,HS,4.0,4.0,CA,0,1,A,40557,40557,...,K,Elementary,K->Elementary,H,1,0.063646,0.936508,4,L,1
1,2,HC,8.0,8.0,AZ,0,7,A,40557,40564,...,Middle,Middle,Middle->Middle,H,1,0.025882,0.880000,3,L,1
2,3,HD,8.0,8.0,FL,0,3,A,40558,40560,...,Middle,Middle,Middle->Middle,H,1,0.025131,0.888889,3,L,1
3,4,HN,9.0,12.0,VA,1,3,B,40558,40560,...,Undefined,Undefined,Undefined->Undefined,H,0,NaN,1.000000,0,NaN,0
4,5,HD,6.0,8.0,FL,0,6,T,40559,40564,...,Middle,Middle,Middle->Middle,H,0,0.112500,0.910112,8,M-L,0


# Paso 4: "Limpiar" los datos

En este caso, los datos se dejan a propósito ligeramente "sucios", es decir, se limpian previamente en cierta medida, pero para efectos de aprendizaje todavía quedan algunos elementos de datos "sucios":

- Algunos campos de datos (variables, características, columnas) tienen tipos incorrectos, por ejemplo, deberían convertirse de números a categorías.

- Algunas variables categóricas tienen demasiados valores (niveles), y algunos de los niveles son demasiado raros: por ejemplo, sólo hay un grupo de Bahamas -- estos datos deberían fusionarse en una categoría más poblada

- Faltan algunos datos y hay que sustituirlos o imputarlos

- Para concluir la limpieza de los datos, tendremos que crear, por supuesto, variables ficticias ("one hot encodig") para las variables categóricas 

In [34]:
# Limpieza de datos -- parte 1: convertir tipos de datos incorrectos

# Algunos de los tipos de datos que maneja Python:
# int -- número entero (e.g., 5)
# float -- número fraccionario (e.g., 5.25)
# object, str -- text (string). Un texto que contiene varios valores no ordenados (e.g., M/F)

df.info() # Para chequear qué tipo de datos tenemos 

# Otros tipos de datos en el paquete pandas:
# category -- categoricos, igual que "factor" in R (e.g., red/green/blue, or M/F: una lista con varios valores no ordenados)
# datetime -- fecha y hora (e.g., 01.01.2020)
# bool -- binario (e.g.? yes/no, 1/0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2389 entries, 0 to 2388
Data columns (total 56 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              2389 non-null   int64  
 1   Program.Code                    2389 non-null   object 
 2   From.Grade                      2262 non-null   float64
 3   To.Grade                        2239 non-null   float64
 4   Group.State                     2389 non-null   object 
 5   Is.Non.Annual.                  2389 non-null   int64  
 6   Days                            2389 non-null   int64  
 7   Travel.Type                     2389 non-null   object 
 8   Departure.Date                  2389 non-null   int64  
 9   Return.Date                     2389 non-null   int64  
 10  Deposit.Date                    2389 non-null   int64  
 11  Special.Pay                     470 non-null    object 
 12  Tuition                         23

In [36]:
# Limpieza de los datos -- parte 1: conversión de tipos de datos que deberían ser categóricos

df['From.Grade'] = df['From.Grade'].astype('category')
df['To.Grade'] = df['To.Grade'].astype('category')
df['Group.State'] = df['Group.State'].astype('category')
df['Is.Non.Annual.'] = df['Is.Non.Annual.'].astype('category')
df['Travel.Type'] = df['Travel.Type'].astype('category')
df['Poverty.Code'] = df['Poverty.Code'].astype('category')
df['CRM.Segment'] = df['CRM.Segment'].astype('category')
df['School.Type'] = df['School.Type'].astype('category')
df['Parent.Meeting.Flag'] = df['Parent.Meeting.Flag'].astype('category')
df['MDR.Low.Grade'] = df['MDR.Low.Grade'].astype('category')
df['MDR.High.Grade'] = df['MDR.High.Grade'].astype('category')
df['School.Sponsor'] = df['School.Sponsor'].astype('category')
df['SchoolGradeTypeLow'] = df['SchoolGradeTypeLow'].astype('category')
df['SchoolGradeTypeHigh'] = df['SchoolGradeTypeHigh'].astype('category')
df['GroupGradeTypeLow'] = df['GroupGradeTypeLow'].astype('category')
df['GroupGradeTypeHigh'] = df['GroupGradeTypeHigh'].astype('category')
df['MajorProgramCode'] = df['MajorProgramCode'].astype('category')
df['SingleGradeTripFlag'] = df['SingleGradeTripFlag'].astype('category')
df['SchoolSizeIndicator'] = df['SchoolSizeIndicator'].astype('category')
df['Retained.in.2012.'] = df['Retained.in.2012.'].astype('category')

df['Region'] = df['Region'].astype('category')
df['Income.Level'] = df['Income.Level'].astype('category')
df['Special.Pay'] = df['Special.Pay'].astype('category')
df['SPR.Product.Type'] = df['SPR.Product.Type'].astype('category')
df['SchoolGradeType'] = df['SchoolGradeType'].astype('category')
df['SPR.New.Existing'] = df['SPR.New.Existing'].astype('category')


df.info() # Chequeemos los resultados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2389 entries, 0 to 2388
Data columns (total 56 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   ID                              2389 non-null   int64   
 1   Program.Code                    2389 non-null   object  
 2   From.Grade                      2262 non-null   category
 3   To.Grade                        2239 non-null   category
 4   Group.State                     2389 non-null   category
 5   Is.Non.Annual.                  2389 non-null   category
 6   Days                            2389 non-null   int64   
 7   Travel.Type                     2389 non-null   category
 8   Departure.Date                  2389 non-null   int64   
 9   Return.Date                     2389 non-null   int64   
 10  Deposit.Date                    2389 non-null   int64   
 11  Special.Pay                     470 non-null    category
 12  Tuition             

In [30]:
df['FPP.to.School.enrollment'].value_counts()

0.08           9
0.04           9
0.1            9
0.066666667    8
0.05           7
              ..
0.011494253    1
0.032547699    1
0.018469657    1
0.088628763    1
0.080838323    1
Name: FPP.to.School.enrollment, Length: 1909, dtype: int64

In [37]:
# Limpieza de datos -- parte 2: combinar categorías poco frecuentes ("niveles")

df['Group.State'].value_counts() # Tomemos el ejemplo de la variable Group.State

# El siguiente es el código para chequear todas las columnas: 
# for col in df.select_dtypes(include=['category','object','bool']).columns:
#    print(col)
#    print(df[col].value_counts())
#    print('\n') 


CA                718
TX                308
WA                147
IL                104
CO                 89
MI                 71
FL                 62
OH                 53
AZ                 53
OR                 51
MN                 51
WI                 46
IN                 43
MO                 43
NE                 42
TN                 38
MA                 36
IA                 35
OK                 33
LA                 31
KS                 26
GA                 22
AL                 21
NV                 20
NM                 20
NY                 19
VA                 18
KY                 16
NC                 16
MD                 15
CT                 15
ID                 14
SD                 11
SC                 10
AR                 10
MS                  9
UT                  9
HI                  9
NH                  7
ME                  7
NJ                  6
MT                  6
ND                  5
PA                  5
AK                  5
MX        

In [38]:

# Presentamos una función personalizada que llamaremos CombineRareCategories
# esta función tiene dos argumentos: el nombre del dataframe (data) y el número mínimo de puntos de datos para seguir 
# siendo una categoría separada (mincount)
# esta función recorrerá todas las columnas del marco de datos y combinará todas las categorías que aparezcan en los 
# datos menos que el número mínimo de veces en (Other)

def CombineRareCategories(data, mincount):
    for col in data.columns:
        if (type(data[col][0]) == str):
            for index, row in pd.DataFrame(data[col].value_counts()).iterrows():
                if ( row[0] < mincount):
                    df[col].replace(index, 'Other_' + col, inplace = True)
                else:
                    None

# Aplicamos esta función a variables con mincount=10                    
CombineRareCategories(df, 10)        

df[0:10] # Chequeamos los resultados

,ID,Program.Code,From.Grade,To.Grade,Group.State,Is.Non.Annual.,Days,Travel.Type,Departure.Date,Return.Date,...,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,SingleGradeTripFlag,FPP.to.School.enrollment,FPP.to.PAX,Num.of.Non_FPP.PAX,SchoolSizeIndicator,Retained.in.2012.
0,1,HS,4.0,4.0,CA,0,1,A,40557,40557,...,K,Elementary,K->Elementary,H,1,0.063646,0.936508,4,L,1
1,2,HC,8.0,8.0,AZ,0,7,A,40557,40564,...,Middle,Middle,Middle->Middle,H,1,0.025882,0.880000,3,L,1
2,3,HD,8.0,8.0,FL,0,3,A,40558,40560,...,Middle,Middle,Middle->Middle,H,1,0.025131,0.888889,3,L,1
3,4,HN,9.0,12.0,VA,1,3,B,40558,40560,...,Undefined,Undefined,Undefined->Undefined,H,0,NaN,1.000000,0,NaN,0
4,5,HD,6.0,8.0,FL,0,6,Other_Travel.Type,40559,40564,...,Middle,Middle,Middle->Middle,H,0,0.112500,0.910112,8,M-L,0
5,6,HC,10.0,12.0,LA,0,4,A,40560,40563,...,High,High,High->High,H,0,0.010650,0.909091,1,L,1
6,7,SG,11.0,12.0,MA,1,6,A,40561,40566,...,High,High,High->High,S,0,0.111111,0.925926,2,S,0
7,8,Other_Program.Code,9.0,9.0,Other_Group.State,0,8,A,40567,40574,...,Undefined,Undefined,Undefined->Undefined,I,1,NaN,0.928571,1,NaN,0
8,9,CC,8.0,8.0,AZ,0,8,A,40572,40579,...,Middle,High,Middle->High,C,1,0.104000,0.928571,4,S-M,1
9,10,HD,8.0,8.0,TX,0,4,A,40581,40584,...,PK,Middle,PK->Middle,H,1,0.103937,0.916667,6,M-L,1


In [39]:
# Limpieza de datos -- parte 3: Reemplazo/Imputación de datos faltantes

pd.DataFrame(df).isna().sum() # Chequeamos si hay datos faltantes

ID                                   0
Program.Code                         0
From.Grade                         127
To.Grade                           150
Group.State                          0
Is.Non.Annual.                       0
Days                                 0
Travel.Type                          0
Departure.Date                       0
Return.Date                          0
Deposit.Date                         0
Special.Pay                       1919
Tuition                              0
FRP.Active                           0
FRP.Cancelled                        0
FRP.Take.up.percent.                 0
Early.RPL                          673
Latest.RPL                          19
Cancelled.Pax                        0
Total.Discount.Pax                   0
Initial.System.Date                  8
Poverty.Code                       599
Region                               0
CRM.Segment                          4
School.Type                          0
Parent.Meeting.Flag      

In [40]:
# Método:
# Variables Categóricas: agregar una nueva categoría 'missing_value' (como si fuera un nuevo color, o género)
# Variables Numéricas: reemplazar con la mediana (o la media, o el valor más frecuente, etc.) Un método alterno es
# ejecutar una imputación, see here: https://scikit-learn.org/stable/modules/impute.html 
# + agregamos columnas sustitutas indicando que el valor ha sido imputado

# creación de variables sustitutas
for col in df:
    if df[col].isna().sum() != 0: 
        df[col + '_surrogate'] = df[col].isna().astype(int)

# fijación de variables categóricas
imputer = SimpleImputer(missing_values = np.nan, strategy='constant')
imputer.fit(df.select_dtypes(exclude=['int64','float64']))
df[df.select_dtypes(exclude=['int64','float64']).columns] = imputer.transform(df.select_dtypes(exclude=['int64','float64']))
           
# fijación de variables numéricas 
imputer = SimpleImputer(missing_values = np.nan, strategy='median')
imputer.fit(df.select_dtypes(include=['int64','float64']))
df[df.select_dtypes(include=['int64','float64']).columns] = imputer.transform(df.select_dtypes(include=['int64','float64']))

# Examinemos los resultados para la variable "Poverty.Code"
df[['Poverty.Code','Poverty.Code_surrogate']]


,Poverty.Code,Poverty.Code_surrogate
0,B,0.0
1,C,0.0
2,C,0.0
3,missing_value,1.0
4,D,0.0
...,...,...
2384,C,0.0
2385,C,0.0
2386,missing_value,1.0
2387,missing_value,1.0


In [ ]:
# Limpieza de datos -- parte 4: creación de dummies para variables no numéricas ("one hot encoding")

df = pd.get_dummies(df, columns = df.select_dtypes(exclude=['int64','float64']).columns, drop_first = True)

pd.options.display.max_columns = None # remove the limit on the number of columns by default only 20 are shows

df.head()  # nuestro dataset tiene ahora 212 columnas (!)

In [ ]:
df.shape

# Paso 5:  Definición del vector objetivo (y) y la matriz de características (X)

In [ ]:
y = df['Retained.in.2012.']
X = df.drop(columns = 'Retained.in.2012.')

# Paso 6:  Dividir X, y en entrenamiento y prueba

In [ ]:
# Definimos la semilla para el generador de número aleatorios
np.random.seed(77300)

# Dividimos los datos aleatoriamente en 80% para entrenamiento y 20% para prueba 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, stratify=y)
# IMPORTANTE: Las muestras están estratificadas, i.e., la proporción de clientes retenidos y no-retenidos es la misma en ambos

# Chequeemos los resultados
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import seaborn as sns
sns.countplot(x=y_test, palette="bright")

# Pasos 7, 8, 9: Desarrollar un modelo con los datos de entrenamiento, Usarlo para predecir los valores en los datos de prueba, Calcular las métricas del modelo, y comparar modelos

In [ ]:
# Primero, definimos un conjunto de funciones para calcular las métricas del modelo

# Curva ROC
def plot_roc(y_test, y_pred):
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1, drop_intermediate = False)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([-0.001, 1.001])
    plt.ylim([-0.001, 1.001])
    plt.xlabel('1-Specificity (False Negative Rate)')
    plt.ylabel('Sensitivity (True Positive Rate)')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

# Matriz de Confusión: cm[0,0], cm[0,1], cm[1,0], cm[1,1]: tn, fp, fn, tp

# Sensitivity
def custom_sensitivity_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tp/(tp+fn))

# Specificity
def custom_specificity_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tn/(tn+fp))

# Positive Predictive Value
def custom_ppv_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tp/(tp+fp))

# Negative Predictive Value
def custom_npv_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tn/(tn+fn))

# Accuracy
def custom_accuracy_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return ((tn+tp)/(tn+tp+fn+fp))

# Modelo № 1: Regresión Logística

Podríamos estar tentados a incluir todas las variables, pero esto haría que el modelo resultante fuera demasiado largo para la selección de variables que sigue. Lo recomendable es realizar un análisis exploratorio de datos donde se apliquen técnicas estadísticas para detectar cuáles variables estarían más relacionadas con la variable dependiente. Por ahora, iniciaremos incluyendo todas las variables.

In [ ]:
# Definimos el modelo y lo llamamos classifier_LR
classifier_LR = LogisticRegression(solver='liblinear')

# Entrenamos classifier_LR con los datos de entrenamiento
classifier_LR.fit(X_train, y_train)

# Esto es una regresión, por tanto tiene coeficientes -- Revisemoslos
# Observe que no hay una manera sencilla de mostrar signficancia, etc. con  sklearn
print('Intercepto: ' + str(classifier_LR.intercept_))
print('Coeficientes (10 mayores and 10 menores) [recuerde, existen 211, en total: ]')
summary = pd.DataFrame([X_test.columns,classifier_LR.coef_[0]]).transpose().sort_values(by = 1, ascending = False)
summary.columns = ['Variable','Coeficiente']
top10positive = summary.head(10) # 10 más grandes (por valor)
top10negative = summary.tail(10) # 10 más pequeños (por valor)
top10list=pd.DataFrame()
top10list= top10list.append(pd.DataFrame(data = top10positive))
top10list= top10list.append(pd.DataFrame(data = top10negative))
top10list


In [ ]:
# Usemos el modelo entrenado para predecir sobre los datos de prueba

y_pred_prob = classifier_LR.predict_proba(X_test)[:,1] # probabilities

# Seleccionamos el valor del umbral -- usaremos Т=0.6073. Porqué? Porqué no 50%? 
# 60.73% -- es la probabilidad promedio de retención en nuestros datos (1451 "1"s de 2389 datos) 
class_threshold = 0.6073

y_pred = np.where(y_pred_prob > class_threshold, 1, 0) # aplicación de la regla del umbral para clasificar

print(y_pred_prob[0:5]) # Primeras 5 probabilidades 
print(y_pred[0:5]) # Resultados pronosticados 
print(y_test[0:5]) # Resultados reales

# oops ... para los primeros 5 clientes nuestro modelo tuvo 2 errores: 
# en el primer cliente ("falso positivo") y en el tercero ("falso negativo")


In [ ]:
# Revisemos las métricas del modelo 
print('Métricas del modelo de regresión logística: \n')

cm = np.transpose(confusion_matrix(y_test, y_pred))
print("Matriz de confusión: \n" + str(cm))

print("                                   Accuracy: " + str(custom_accuracy_score(y_test, y_pred))) 
print("                       SENSITIVITY (RECALL): " + str(custom_sensitivity_score(y_test, y_pred)))
print("                     SPECIFICITY (FALL-OUT): " + str(custom_specificity_score(y_test, y_pred)))
print("     POSITIVE PREDICTIVE VALUE, (PRECISION): " + str(custom_ppv_score(y_test, y_pred)))
print("                  NEGATIVE PREDICTIVE VALUE: " + str(custom_npv_score(y_test, y_pred)))

plot_roc(y_test, y_pred_prob)
print(" AUC: " + str(roc_auc_score(y_test, y_pred_prob)))


## Interpretación de la curva ROC y el AUC: 

- Se sabe que el modelo está cometiendo alrededor del 34% de error, pero este error es uniforme para todos los clientes? Sería algo no deseable puesto que el moodelo no será tan decisivo. Si tomamos un cliente con una probabilidad pronósticada de 99%, uno con una probabilidad de 50%, y uno con una probabilidad del 1%. Esperaríamos que el modelo nos diera una certeza de que el primer cliente será retenido, y una certeza de que el tercero no lo será, pero podríamos estar cómodos con un error grande para el segundo.

- La curva ROC muestra justamente eso. Ranquea todos los clientes desde el que tiene la probabilidad más alta de retención hasta la más baja (equivale a variar el umbral desde alto hasta bajo). Comenzando en el origen, mapea todos los clientes en orden descendiente de probabilidad (desde el “mejor” hasta el “peor”). Un clasificador perfecto, con exactitud 100%, primero predeciría correctamente todos los positivos, y luego predeciría correctamente todos los negativos; es decir, la curva iría recto hasta el punto (0,1), y luego cambiaría y sería horizontal hasta el punto (1,1). Esto, por supuesto, no es posible en la práctica, y los “pasos” en la curva reflejan los errores ocasionales que el modelo comete. Un buen modelo cometería pocos errores positivos para los mejores clientes y pocos eroreres negativos para los peores.

-  Es importante observar que un modelo que simplemente adivina al azar, tendrá como curva ROC una línea de 45 grados. Tal modelo tendría la misma probabilidad de hacer una predicción correcta que una incorrecta, sin importar si el cliente tiene una alta o baja probabilidad predecida.

- En este caso el AUC es 83.12% el cual no es un mal resultado. El AUC indica la proporción de parejas concordantes en los datos; en este caso el porcentaje de parejas concordantes es aproximadamente 83%, lo cual es bueno. Las parejas concordantes son aquellas parejas de casos positivo y negativo en el dataset para las cuales el modelo SVM - con ciertos parámetros - puede clasificarlos correctamente.

- En el dataset de prueba, tenemos 290 positivos (clientes retenidos) y 188 negativos (clientes no retenidos); el número total de parejas (positivos y negativos) es 290 x 188 = 54520, de los cuales 73.16% (= 39887) tienen unos parámetros del SVM que pueden clasificarlos correctamente.

In [ ]:
# MÉTODO RFE

# Aplicamos selección de variables con Stepwise Recursive Feature Selection
# El método Recursive Feature Elimination (RFE) es un método para selección de variables.  Funciona removiendo recursivamente
# atributos y construye un modelo sobre los atributos restantes.  Usa el accuracy del modelo para identificar cuáles atributos
# (y combinación de atributos) contribuyen más significativamente a la predicción del atributo objetivo.

from sklearn.feature_selection import RFE

rfe = RFE(estimator=classifier_LR, n_features_to_select=20, step=1) # in this example we will select 20 variables; this number "20" is a hyperparameter to tune
rfe.fit(X_train, y_train)
ranking = rfe.ranking_.reshape(len(X_train.columns))

# Cuáles son las 20 variables que quedan en el modelo?
pd.DataFrame([X_test.columns,ranking]).transpose().sort_values(1).head(30)


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(ranking)
print(rfe.get_support())

In [ ]:
# Entrenemos el nuevo modelo y llamemoslo classifier_LR_RFE 
classifier_LR_RFE = rfe.fit(X_train, y_train)

# Usemos el modelo entrenado para predecir sobre los datos de prueba
y_pred_prob = classifier_LR_RFE.predict_proba(X_test)[:,1] # probabilidades
y_pred = np.where(y_pred_prob > class_threshold, 1, 0) # clasificación


# Revisemos las métricas del modelo después de la selección de variables 
print('Métricas del modelo de regresión logística después de la selección de variables: \n')

cm = np.transpose(confusion_matrix(y_test, y_pred))
print("Matriz de confusión: \n" + str(cm))

print("                                   Accuracy: " + str(custom_accuracy_score(y_test, y_pred))) 
print("                       SENSITIVITY (RECALL): " + str(custom_sensitivity_score(y_test, y_pred)))
print("                     SPECIFICITY (FALL-OUT): " + str(custom_specificity_score(y_test, y_pred)))
print("     POSITIVE PREDICTIVE VALUE, (PRECISION): " + str(custom_ppv_score(y_test, y_pred)))
print("                  NEGATIVE PREDICTIVE VALUE: " + str(custom_npv_score(y_test, y_pred)))

plot_roc(y_test, y_pred_prob)
print(" AUC: " + str(roc_auc_score(y_test, y_pred_prob)))


## Resumen de la Regresión Logística: 

- Primero entrenamos el modelo con el conunto completo de 211 variables
- Cuando lo aplicamos a los datos de prueba, AUC=73,2%
- Con la selección de variables, AUC aumentó a 85,4%

# Modelo № 2: Árboles de decisión

In [ ]:
# Definimos un modelo CART y los llamamos classifier_DT
classifier_DT = DecisionTreeClassifier(max_leaf_nodes = 5, random_state=77300) 

# Entrenamos el modelo classifier_DT con los datos de entrenamiento
classifier_DT.fit(X_train, y_train)

# Usamos el modelo entrenado para predecir sobre los datos de prueba
y_pred_prob = classifier_DT.predict_proba(X_test)[:,1] # probabilidades 
y_pred = np.where(y_pred_prob > class_threshold, 1, 0) # clasificación

print(y_pred_prob[0:5]) # primeras 5 probabilidades 
print(y_pred[0:5]) # resultados pronosticados 
print(y_test[0:5]) # resultados reales

# WOW -- el mnodelo CART no comete errores sobre los primeros 5 clientes! 


In [ ]:
# Visualizaicón del árbol resultante

from sklearn import tree

plt.figure(figsize=(40,20))
tree.plot_tree(classifier_DT.fit(X_train, y_train), feature_names = X_train.columns, filled = True, 
               class_names = ['No-Retenido', 'Retenido'], rounded = True)
print('árbol CART con 5 nodos terminales')


In [ ]:
# Revisemos las métricas del modelo

print('Métricas del modelo CART: \n')

cm = np.transpose(confusion_matrix(y_test, y_pred))
print("Matriz de Confusión: \n" + str(cm))

print("                                   Accuracy: " + str(custom_accuracy_score(y_test, y_pred))) 
print("                       SENSITIVITY (RECALL): " + str(custom_sensitivity_score(y_test, y_pred)))
print("                     SPECIFICITY (FALL-OUT): " + str(custom_specificity_score(y_test, y_pred)))
print("     POSITIVE PREDICTIVE VALUE, (PRECISION): " + str(custom_ppv_score(y_test, y_pred)))
print("                  NEGATIVE PREDICTIVE VALUE: " + str(custom_npv_score(y_test, y_pred)))

plot_roc(y_test, y_pred_prob)
print(" AUC: " + str(roc_auc_score(y_test, y_pred_prob)))

In [ ]:
# Tuning de hiper-parámetros. Un modelo CART tiene múltiples hiper-parámetros, por ejemplo:
# -- máximo número de nodos terminales (hojas) en un árbol, 
# -- mínimo número de datos en un nodo terminal (hoja)
# -- mínimo número de datos para crear una ramificación (split)
# y otros

DecisionTreeClassifier() # desplegamos cuáles son estos hiper-parámetros y sus valores por defecto

In [ ]:
# Tuning del hiper-parámetro max_leaf_nodes

n_max_leaf_nodes = range(5,40) # Entrenamos los modelos con 5, 6, 7, ... 40 hojas

# para cada modelo calculamos el AUC para prueba 
array = []
for n in n_max_leaf_nodes:
    
    classifier_DT = tree.DecisionTreeClassifier(criterion = 'gini', max_leaf_nodes = n)
    classifier_DT = classifier_DT.fit(X_train, y_train)
    
    y_pred_prob = classifier_DT.predict_proba(X_test)[:,1]   
    y_pred = np.where(y_pred_prob > class_threshold, 1, 0)

    array.append([n,roc_auc_score(y_test, y_pred_prob)])

# graficamos los AUCs sobre el dataset de prueba
array = pd.DataFrame(array)
plt.scatter(array[0],array[1])

# ahora, para cada modelo calculamos AUC sobre el dataset de entrenamiento 
array = []
for n in n_max_leaf_nodes:

    classifier_DT = tree.DecisionTreeClassifier(criterion = 'gini', max_leaf_nodes = n)
    classifier_DT = classifier_DT.fit(X_train, y_train)
    
    y_pred_prob = classifier_DT.predict_proba(X_train)[:,1] 
    y_pred = np.where(y_pred_prob > class_threshold, 1, 0)

    array.append([n,roc_auc_score(y_train, y_pred_prob)])

# graficamos los AUCs sobre el dataset de entrenamiento
array = pd.DataFrame(array)
plt.scatter(array[0],array[1])

# etiquetamos los ejes en el gráfico
plt.xlabel('Hiper-parámetro: max_leaf_nodes')
plt.ylabel('AUC')

# add the legend
plt.legend(['Dataset de Prueba','Dataset de Entrenamiento'])


## Este gráfico ilustra el concepto de "overfitting":  

- entre más hojas tenga el árbol, más alto es el AUC sobre el dataset de entrenamiento

- sin embargo, comenzando a partir de ~8-12 hojas, el AUC de prueba comienza a disminuir, puesto que un modelo que es demasiado compleo "aprende" algo que no generaliza a los nuevos datos

In [ ]:
# entrenemos el modelo con 8 hojas
classifier_DT = tree.DecisionTreeClassifier(criterion = 'gini', max_leaf_nodes = 8)
classifier_DT = classifier_DT.fit(X_train, y_train)

# obtenemos sus predicciones
y_pred_prob = classifier_DT.predict_proba(X_test)[:,1]   
y_pred = np.where(y_pred_prob > class_threshold, 1, 0)

# calculamos e imprimimos el AUC
print(" AUC: " + str(roc_auc_score(y_test, y_pred_prob)))

## Resumen del CART: 

- primero, entrenamos el modelo con los hiper-parámetros por defecto
- aplicandolo al dataset de prueba, obtuvimos AUC=80.0%
- luego, sintonizamos el parámetro max_leaf_nodes y el AUC aumentó a 83.8%

# Modelo № 3: Support Vector Machines

In [ ]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

svm_estimators = []
svm_estimators.append(('standardize', StandardScaler())) # escalamos los datos
svm_estimators.append(('svm', svm.SVC(probability=True))) # definimos SVM con probabilidades 
     
# Definimos el modelo SVM y lo llamamos classifier_SVM
Classifier_SVM = Pipeline(svm_estimators, verbose=False)

# Entrenamos el modelo classifier_SVM sobre los datos de entrenamiento
Classifier_SVM.fit(X_train, y_train)

In [ ]:
# Usamos el modelo desarrollado, para predecir sobre los datos de prueba 
y_pred_prob = Classifier_SVM.predict_proba(X_test)[:,1] # probabilidades
y_pred = np.where(y_pred_prob > class_threshold, 1, 0) # clasificación

# Revisemos las métricas del modelo

print('Métricas del modelo de Máquina de Vectores de Soporte: \n')

cm = np.transpose(confusion_matrix(y_test, y_pred))
print("Matriz de Confusión: \n" + str(cm))

print("                                   Accuracy: " + str(custom_accuracy_score(y_test, y_pred))) 
print("                       SENSITIVITY (RECALL): " + str(custom_sensitivity_score(y_test, y_pred)))
print("                     SPECIFICITY (FALL-OUT): " + str(custom_specificity_score(y_test, y_pred)))
print("     POSITIVE PREDICTIVE VALUE, (PRECISION): " + str(custom_ppv_score(y_test, y_pred)))
print("                  NEGATIVE PREDICTIVE VALUE: " + str(custom_npv_score(y_test, y_pred)))

plot_roc(y_test, y_pred_prob)
print(" AUC: " + str(roc_auc_score(y_test, y_pred_prob)))

In [ ]:
###  Ejecución de SVM con las 20 variables seleccionadas del dataset A

In [ ]:
### Con el mejor modelo (todas o solo 20 variables del dataset A), desarrollar SVM lineal sintonizando hiper-parámetros

In [ ]:
### Con el mejor modelo (todas o solo 20 variables del dataset A), desarrollar SVM radial sintonizando hiper-parámetros

In [ ]:
### Agregar dataset B al dataset A = dataset completo.  Dividir en entrenamiento y prueba.  

In [ ]:
### Desarrollar modelo de Regresión Logística con todas las variables del dataset completo, y evaluar su desempeño

In [ ]:
### Desarrollar modelo de Regresión Logística seleccionando las 20 variables más importantes del dataset completo, 
### y evaluar su desempeño

In [ ]:
###  Ejecución de SVM con todas las variables del dataset completo, y evaluar desempeño

In [ ]:
###  Ejecución de SVM con las 20 variables seleccionadas del dataset completo, y evaluar desempeño

In [ ]:
### Con el mejor modelo (todas o solo 20 variables del dataset A), desarrollar SVM lineal sintonizando hiper-parámetros

In [ ]:
### Con el mejor modelo (todas o solo 20 variables del dataset A), desarrollar SVM radial sintonizando hiper-parámetros

## Resumen del SVM: 

- entrenamos el modelo con los hiper-parámetros por defecto
- aplicando el modelo al dataset de prueba, obtuvimos AUC=85.7%
- ahora, sintonizaremos los hiper-parámetros del modelo ...

# Paso 10: Exportar las predicciones del mejor modelo, para uso posterior

In [ ]:
# Si tenemos los datos de predicción (acerca de los nuevos clientes), primero los cargamos y limpliamos para obtener la
# matriz exactamente con la misma estructura de X_train
# Como no tenemos tales datos, pronosticaremos los datos de prueba y exportaremos las predicciones resultantes

X_pred = X_test

# Cuál modelo deberíamos tomar? 

# En estge momento, el mejor modelo es el SVM con hiper-parámetros optimizados.  
y_pred_prob = grid_search_SVM.predict_proba(X_pred)[:,1]   

# Lets add the ID column to know "who is who"
Prediction = pd.DataFrame(data={"ID":X_pred["ID"],"Predicted Probability":y_pred_prob}) 

# Export the predictions into a CSV file
Prediction.to_csv("Predicted Retention Probability_testing.csv",sep = ',')

# BONUS: Gráfico de Ganancias ("Lift")

In [ ]:
# pip install scikit-plot

import scikitplot as skplt

In [ ]:
# as an example, here is the gains chart with SVM; note, people often call this "lift chart"
# note, this is a "sister-plot" to the lift chart we saw in class -- take the ratio of the Class 1 and Baseline values to obtain what we saw

# La curva de ganancias acumuladas es una curva de evaluación que evalúa el rendimiento del modelo y compara los 
# resultados con la selección aleatoria. Muestra el porcentaje de targets logrados al considerar un determinado 
# porcentaje de la población con la mayor probabilidad de ser el target de acuerdo con el modelo.

y_pred_proba=Classifier_SVM.predict_proba(X_test)

skplt.metrics.plot_cumulative_gain(np.int32(y_test), y_pred_proba)
plt.ylabel("Cumulative Gain Value")
plt.xlabel("Rate of Predictions")
plt.title("Cumulative Gain Chart")
plt.show()